# IR Lab SoSe 2024: Erweitertes Retrieval System mit BM25 und TF-IDF

Dieses Jupyter Notebook dient als erweitertes Retrieval-System, das sowohl BM25 als auch TF-IDF verwendet.
Wir verwenden ein Korpus wissenschaftlicher Arbeiten (Titel + Abstracts) aus den Bereichen Information Retrieval und Natural Language Processing (die [IR Anthology](https://ir.webis.de/anthology/) und die [ACL Anthology](https://aclanthology.org/)). Dieses Jupyter Notebook dient nur als Retrieval-System, d.h. es erhält eine Reihe von Informationsbedürfnissen (Topics) und ein Korpus als Eingabe und erzeugt eine Run-Datei als Ausgabe. Bitte führen Sie Auswertungen in einem neuen, dedizierten Notebook durch.

### Schritt 1: Bibliotheken importieren

Wir verwenden [tira](https://www.tira.io/), eine Information Retrieval Shared Task Plattform, zum Laden des (vorgebauten) Retrieval-Index und [ir_dataset](https://ir-datasets.com/), um anschließend ein Retrieval-System mit [PyTerrier](https://github.com/terrier-org/pyterrier), einer Open-Source-Suchmaschine, aufzubauen.

In [ ]:
# Sie müssen diese Zelle nur ausführen, wenn Sie Google Colab verwenden.
# Wenn Sie GitHub Codespaces verwenden, ist alles bereits installiert.
!pip3 install tira ir-datasets python-terrier

In [ ]:
# Imports
from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run
from tira.rest_api_client import Client
import pyterrier as pt
from pyterrier.pipelines import *

In [ ]:
# Erstellen Sie einen REST-Client für die TIRA-Plattform, um auf die vorindexierten Daten zuzugreifen.
ensure_pyterrier_is_loaded()
tira = Client()

### Schritt 2: Dataset und Index laden

In [ ]:
# Das Dataset: die Vereinigung der IR Anthology und der ACL Anthology
pt_dataset = pt.get_dataset('irds:ir-lab-sose-2024/ir-acl-anthology-20240504-training')

# Ein (vorgebauter) PyTerrier-Index, geladen von TIRA
index = tira.pt.index('ir-lab-sose-2024/tira-ir-starter/Index (tira-ir-starter-pyterrier)', pt_dataset)

### Schritt 3: Erweiterte Retrieval Pipeline definieren

Wir definieren eine Retrieval-Pipeline, die sowohl BM25 als auch TF-IDF verwendet.

In [ ]:
# BM25 und TF-IDF Retrieval-Modelle definieren
bm25 = pt.BatchRetrieve(index, wmodel="BM25")
tfidf = pt.BatchRetrieve(index, wmodel="TF_IDF")

# Kombinierte Pipeline: BM25 gefolgt von TF-IDF Re-Ranking
combined_pipeline = bm25 >> tfidf

### Schritt 4: Run erstellen

In [ ]:
print('Zunächst werfen wir einen kurzen Blick auf die ersten drei Topics:')
pt_dataset.get_topics('text').head(3)

In [ ]:
print('Jetzt führen wir das Retrieval durch...')
run = combined_pipeline(pt_dataset.get_topics('text'))

print('Fertig. Hier sind die ersten 10 Einträge des Runs')
run.head(10)

### Schritt 5: Run-Datei für nachfolgende Auswertungen persistieren

In [ ]:
persist_and_normalize_run(run, system_name='bm25-tfidf-combined', default_output='../runs')